In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import re

In [7]:
df = pd.read_csv('New Construction Miami Developments - Brickell.csv')

In [8]:
df

,address,developer,size
0,"1420 South Miami Avenue, Miami, FL 33130",Mast Capital,397 units
1,"90 Southwest Eighth Street, Miami, FL 33130",Gazit Globe Group,504 units
2,"1809 Brickell Avenue, Miami, FL",Related Group & Integra Investments,354 units
3,"99 Southwest Seventh Street, Miami, FL",Harvey Hernandez,3 towers


In [9]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [10]:
df['loc'] = df['address'].apply(geolocator.geocode, timeout=10)

In [11]:
df['loc']

0    (1420 S Miami Ave, Miami, FL 33130, USA, (25.7...
1    (90 SW 8th St, Miami, FL 33130, USA, (25.76606...
2    (1809 Brickell Ave, Miami, FL 33129, USA, (25....
3    (99 SW 7th St, Miami, FL 33130, USA, (25.76748...
Name: loc, dtype: object

In [12]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [13]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,address,developer,size,loc,point,lat,lon,altitude
0,"1420 South Miami Avenue, Miami, FL 33130",Mast Capital,397 units,"(1420 S Miami Ave, Miami, FL 33130, USA, (25.7...","(25.76014, -80.1939079, 0.0)",25.760140,-80.193908,0.0
1,"90 Southwest Eighth Street, Miami, FL 33130",Gazit Globe Group,504 units,"(90 SW 8th St, Miami, FL 33130, USA, (25.76606...","(25.7660695, -80.1949727, 0.0)",25.766070,-80.194973,0.0
2,"1809 Brickell Avenue, Miami, FL",Related Group & Integra Investments,354 units,"(1809 Brickell Ave, Miami, FL 33129, USA, (25....","(25.7542214, -80.1950473, 0.0)",25.754221,-80.195047,0.0
3,"99 Southwest Seventh Street, Miami, FL",Harvey Hernandez,3 towers,"(99 SW 7th St, Miami, FL 33130, USA, (25.76748...","(25.7674854, -80.19533059999999, 0.0)",25.767485,-80.195331,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
#df.at[1,'Buyer Agent']=('Dora Puig, Carl Gambino')

In [14]:
df.columns

Index(['address', 'developer', 'size', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [17]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=15)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Brickell Development Projects')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["developer"], r["size"], r["address"])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Developer: {bold_start}{info[0]}{bold_end}{new_line} \
    Detail: {bold_start}{info[1]}{bold_end}{new_line}Address: \
    {bold_start}{info[2]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))

m

In [18]:
m.save('index.html')